In [ ]:
from google.colab import drive
from bs4 import BeautifulSoup
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install langdetect
!pip install translate
from translate import Translator
from langdetect import detect


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=516c89eb795c5b81c85837baee2659071d0db789d37a8ec03f1b4dbd51f78b8a
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/IR/Project/Job_Posting_dataset.csv")
test_data = pd.read_csv("/content/drive/My Drive/IR/Project/testing_dataset.csv")
test_data

,LinkedIn Resume,Job Profiles
0,Shyvee Shi\n(She/Her)\n3rd degree connection\n...,"['Product Manager','Business Analyst','UX Desi..."
1,Elisa Bellagamba\n\nAbout\nHigh-impact product...,"['Product Manager','Business Analyst','Marketi..."
2,Joni (Rafalski) Hoadley (She/Her)\n\nAbout\nI’...,"['Product Manager','UX Designer','Business Ana..."
3,Shane Connelly\n\nAbout\nI lead the product ma...,"['Product Manager','Software Developer,'Machin..."
4,Dana Tom\n(She/Her)\n\nAbout\nI’m a product ma...,"['Product Manager','Marketing Analyst','Machin..."
...,...,...
367,Johanna P.\n\nAbout\nExperienced Bilingual Rep...,"['Sales Analyst','Marketing Analyst,''Business..."
368,Rachel Sedlacek\n\nAbout\nAnalyst with 12 year...,"['Sales Analyst','Marketing Analyst,''Business..."
369,Deirdre Derby\n\nAbout\n[]\n\nLocation\nMinnea...,"['Sales Analyst','Marketing Analyst,''Business..."
370,Ryan Mitchell\n\nAbout\nSr. Sales Analyst for ...,"['Sales Analyst','Marketing Analyst,''Business..."


In [ ]:
#@title Removing all the rows that contain description in other languages than English
for index, row in data.iterrows():
    try:
        lang = detect(row['Job_Description'])
        if lang != 'en':
            data.drop(index, inplace=True)
    except:
        # if an error occurs, assume the language is not English and drop the row
        data.drop(index, inplace=True)

In [ ]:
data.reset_index(inplace=True)
data

,index,Job ID,Job Title,Company,Job_Description,Job Profile
0,1,3489403427,Software Engineer,LinkedIn,"The ideal candidate will help build, maintain,...",Software Developer
1,2,3490979195,Software Engineer,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",Software Developer
2,3,3507663809,Junior Software Developer (Web/Front-End),Samsung Brasil,Position Summary\n\n\n\n\nDevelop a differenti...,Software Developer
3,4,3497871312,Software Engineer,Oracle,Want to come join the Oracle Health Data & Ana...,Software Developer
4,6,3497654432,Software Engineer,Illuma,We are looking for a highly motivated Software...,Software Developer
...,...,...,...,...,...,...
909,985,3496041048,Marketing Analyst,TI Fluid Systems,Description\n\nDescription:\n\nPosition Summar...,Marketing Analyst
910,986,3496037782,Marketing Analyst,TI Fluid Systems,Description\n\nDescription:\n\nPosition Summar...,Marketing Analyst
911,987,3485523798,Marketing Analyst,Hose Master LLC,POSITION SUMMARY\n\nThe Marketing Analyst is r...,Marketing Analyst
912,988,3488196177,Senior Marketing Analyst,Maxeon Solar Technologies,Maxeon Solar Technologies is a global leader i...,Marketing Analyst


In [ ]:
data['Job Profile'].value_counts()

Machine Learning Engineer    145
Product Manager              143
Marketing Analyst            138
UX Designer                  106
Account Manager              100
Sales Analyst                 99
Business Analyst              98
Software Developer            85
Name: Job Profile, dtype: int64

In [ ]:
str1 = "Raghav Bhalla is the name"
str1 = str1.replace("Raghav Bhalla","")
str1

' is the name'

In [ ]:
for index, row in data.iterrows():
  job_desc_ith = data.loc[index,'Job_Description']
  job_desc_ith = job_desc_ith.replace("Software Developer","")
  job_desc_ith = job_desc_ith.replace("Machine Learning Engineer","")
  job_desc_ith = job_desc_ith.replace("Product Manager","")
  job_desc_ith = job_desc_ith.replace("Marketing Analyst","")
  job_desc_ith = job_desc_ith.replace("UX Designer","")
  job_desc_ith = job_desc_ith.replace("Account Manager","")
  job_desc_ith = job_desc_ith.replace("Sales Analyst","")
  job_desc_ith = job_desc_ith.replace("Business Analyst","")
  data.loc[index,'Job_Description'] = job_desc_ith


# save the cleaned dataframe
data.to_csv('/content/drive/My Drive/IR/Project/english_training_data.csv', index=False)



In [ ]:
def detect_lang(text):
  language = detect(text)
  return language

In [ ]:
def to_Eng(text):
  translator= Translator(to_lang="en")
  translation = translator.translate(text)
  return translation

In [ ]:
def get_tokenizeText(text):
    word_tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

In [ ]:
def vectorize(input_text):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(input_text)
  return X.toarray()

In [ ]:
import string
def remove_punctuation(input_string):
    punctuation = set(string.punctuation)
    no_punct = ''.join(char for char in input_string if char not in punctuation)
    return no_punct


In [ ]:
#@title (No need to run) This cell can be edit if we will try to convert job_description to english
job_des = data["Job_Description"]
print(len(job_des))
all_Des = []
for i in job_des:
  if(detect_lang(i)=='en'):
    # temp = to_Eng(i)
    all_Des.append(i)
    # print(job_des[i])
    # print(temp)
  # else:
  #   all_Des.append(i)

  # print(type(detect_lang("my name is zaid and I am 22 year old, I live in delhi india")))
print(len(all_Des))

914
914


In [ ]:
tokenized_des = []
job_des = data["Job_Description"]
for i in job_des:

  # converting to lowercase
  i = i.replace("\n","")
  lowered_text = i.lower()

  # rempve punctuation
  without_punctuation = remove_punctuation(lowered_text)

  # Tokenized text
  tokenized_text = get_tokenizeText(without_punctuation)
  # print(tokenized_text)
  # ============================== #

  vectorized_description =  vectorize(tokenized_text)
  tokenized_des.append(vectorized_description)

  # # vectorize the text
  # print(vectorize(without_punctuation))

In [ ]:
for i in tokenized_des:
  for j in i:
    print(j)
# ===== Vectorization ========== #

# vectorized_description =  vectorize(tokenized_des)
# for i in vectorized_description:
  # for j in i:
    # print(j)
# print(vectorized_description)
# for i in vectorized_description:
#   print(i)

In [ ]:
data_cpy = data

#df_test = pd.DataFrame()
#df_test['Job_Description'] = test_data['LinkedIn Resume']


In [ ]:
#data_cpy = data_cpy.append(df_test)
#data_cpy

,index,Job ID,Job Title,Company,Job_Description,Job Profile
0,1.0,3.489403e+09,Software Engineer,LinkedIn,"The ideal candidate will help build, maintain,...",Software Developer
1,2.0,3.490979e+09,Software Engineer,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",Software Developer
2,3.0,3.507664e+09,Junior Software Developer (Web/Front-End),Samsung Brasil,Position Summary\n\n\n\n\nDevelop a differenti...,Software Developer
3,4.0,3.497871e+09,Software Engineer,Oracle,Want to come join the Oracle Health Data & Ana...,Software Developer
4,6.0,3.497654e+09,Software Engineer,Illuma,We are looking for a highly motivated Software...,Software Developer
...,...,...,...,...,...,...
367,NaN,NaN,NaN,NaN,johanna p. experienc bilingu repres demonstr h...,NaN
368,NaN,NaN,NaN,NaN,rachel sedlacek analyst 12 year background sup...,NaN
369,NaN,NaN,NaN,NaN,deirdr derbi locat minneapoli minnesota unit s...,NaN
370,NaN,NaN,NaN,NaN,ryan mitchel sr. sale analyst zarbe ’ na sale ...,NaN


In [ ]:
nltk.download('all')


In [ ]:
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
curr_sent = "My name is Raghav. I am running. I love cooking. Raghav is a good boy. boys He cooks good food, he is named after him I have more cousins."
new_sent = []
curr_sent=curr_sent.lower()
  #tokenizing into words
tokens = nltk.word_tokenize(curr_sent)
  #removing stop words
words=[]
for word in tokens:
  if word not in stopwords.words('english'):
    words.append(word)
  #removing punctuation
_words = list(filter(lambda token: token not in string.punctuation, words))
  #removing blank space
words_ = list(filter(lambda token: token not in [' '], _words))

for word in words_:
 new_sent.append(lemmatizer.lemmatize(word))
for word in words_:
  new_sent.append(ps.stem(word))
new_sent = TreebankWordDetokenizer().detokenize(new_sent)
print(new_sent)

name raghav run love cook raghav good boy boy cook good food name cousin


In [ ]:
X1 = cv.transform([curr_sent])

In [ ]:
features= pd.DataFrame(X1.toarray(),columns=cv.get_feature_names_out())
features

,00,000,00005466,000to,001,00am,00pm,01,012023,02,...,zone,zoom,zoominfo,zrx,zug,zyhvz7iigr,zyuiom_ufdu,überraschung,œ100,œmost
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [ ]:
for index,row in data_cpy.iterrows():
  new_sent = []
  curr_sent = row['Job_Description']
  #lowering the sentence
  curr_sent=curr_sent.lower()
  #tokenizing into words
  tokens = nltk.word_tokenize(curr_sent)
  #removing stop words
  words=[]
  for word in tokens:
    if word not in stopwords.words('english'):
      words.append(word)
  #removing punctuation
  _words = list(filter(lambda token: token not in string.punctuation, words))
  #removing blank space
  words_ = list(filter(lambda token: token not in [' '], _words))
  #lemmatization
  for word in words_:
   new_sent.append(lemmatizer.lemmatize(word))
  #for word in words_:
   # new_sent.append(ps.stem(word))
  new_sent = TreebankWordDetokenizer().detokenize(new_sent)
  data_cpy.loc[index,'Job_Description'] = new_sent
  

In [ ]:
for index,row in test_data.iterrows():
  new_sent = []
  curr_sent = row['LinkedIn Resume']
  #lowering the sentence
  curr_sent=curr_sent.lower()
  #tokenizing into words
  tokens = nltk.word_tokenize(curr_sent)
  #removing stop words
  words=[]
  for word in tokens:
    if word not in stopwords.words('english'):
      words.append(word)
  #removing punctuation
  _words = list(filter(lambda token: token not in string.punctuation, words))
  #removing blank space
  words_ = list(filter(lambda token: token not in [' '], _words))
  #lemmatization
  for word in words_:
    new_sent.append(lemmatizer.lemmatize(word))
  #for word in words_:
   # new_sent.append(ps.stem(word))
  new_sent = TreebankWordDetokenizer().detokenize(new_sent)
  test_data.loc[index,'Job_Description'] = new_sent

In [ ]:
data_cpy

,index,Job ID,Job Title,Company,Job_Description,Job Profile
0,1,3489403427,Software Engineer,LinkedIn,ideal candid help build maintain troubleshoot ...,Software Developer
1,2,3490979195,Software Engineer,PayPal,paypal nasdaq pypl believ everi person right p...,Software Developer
2,3,3507663809,Junior Software Developer (Web/Front-End),Samsung Brasil,posit summari develop differenti system unders...,Software Developer
3,4,3497871312,Software Engineer,Oracle,want come join oracl health data analyt team l...,Software Developer
4,6,3497654432,Software Engineer,Illuma,look highli motiv softwar engin work within te...,Software Developer
...,...,...,...,...,...,...
909,985,3496041048,Marketing Analyst,TI Fluid Systems,descript descript posit summari market analyst...,Marketing Analyst
910,986,3496037782,Marketing Analyst,TI Fluid Systems,descript descript posit summari market analyst...,Marketing Analyst
911,987,3485523798,Marketing Analyst,Hose Master LLC,posit summari respons conduct market analysi s...,Marketing Analyst
912,988,3488196177,Senior Marketing Analyst,Maxeon Solar Technologies,maxeon solar technolog global leader solar inn...,Marketing Analyst


In [ ]:
cv = CountVectorizer()

X = cv.fit_transform(data_cpy['Job_Description'])


In [ ]:
X

<914x9698 sparse matrix of type '<class 'numpy.int64'>'
	with 186676 stored elements in Compressed Sparse Row format>

In [ ]:
X.shape

(914, 9698)

In [ ]:
print(cv.vocabulary_)

{'ideal': 4420, 'candid': 1599, 'help': 4201, 'build': 1506, 'maintain': 5366, 'troubleshoot': 8910, 'compani': 2019, 'rapidli': 7124, 'expand': 3297, 'infrastructur': 4586, 'work': 9549, 'alongsid': 703, 'engin': 3101, 'ensur': 3119, 'highest': 4236, 'level': 5148, 'perform': 6510, 'abil': 460, 'manag': 5387, 'problem': 6857, 'aris': 900, 'respons': 7365, 'monitor': 5744, 'instal': 4632, 'system': 8486, 'test': 8650, 'oper': 6226, 'applic': 856, 'qualif': 7044, 'bachelor': 1095, 'degre': 2536, 'comput': 2065, 'scienc': 7663, 'relat': 7261, 'field': 3480, 'experi': 3304, 'configur': 2098, 'strong': 8332, 'script': 7683, 'technic': 8579, 'skill': 7957, 'show': 7874, 'paypal': 6461, 'nasdaq': 5886, 'pypl': 7021, 'believ': 1204, 'everi': 3237, 'person': 6531, 'right': 7447, 'particip': 6418, 'fulli': 3758, 'global': 3915, 'economi': 2954, 'mission': 5675, 'democrat': 2568, 'financi': 3501, 'servic': 7785, 'everyon': 3242, 'regardless': 7235, 'background': 1101, 'econom': 2952, 'stand': 82

In [ ]:
data_features= pd.DataFrame(X.toarray(),columns=cv.get_feature_names_out())
data_features

,00,000,00005466,000to,001,00am,00pm,01,012023,02,...,zone,zoom,zoominfo,zrx,zug,zyhvz7iigr,zyuiom_ufdu,überraschung,œ100,œmost
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
911,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
912,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test = cv.transform(test_data['LinkedIn Resume'])

In [ ]:
X_test

<372x9698 sparse matrix of type '<class 'numpy.int64'>'
	with 63797 stored elements in Compressed Sparse Row format>

In [ ]:
X_test.shape

(372, 9698)

In [ ]:
test_features= pd.DataFrame(X_test.toarray(),columns=cv.get_feature_names_out())
test_features

,00,000,00005466,000to,001,00am,00pm,01,012023,02,...,zone,zoom,zoominfo,zrx,zug,zyhvz7iigr,zyuiom_ufdu,überraschung,œ100,œmost
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_features['Job_Profile'] = data['Job Profile']
data_features

,00,000,00005466,000to,001,00am,00pm,01,012023,02,...,zoom,zoominfo,zrx,zug,zyhvz7iigr,zyuiom_ufdu,überraschung,œ100,œmost,Job_Profile
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Software Developer
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Software Developer
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Software Developer
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Software Developer
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Software Developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Marketing Analyst
910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Marketing Analyst
911,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Marketing Analyst
912,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Marketing Analyst


In [ ]:
test_features['Job Profiles'] = test_data['Job Profiles']
test_features

,00,000,00005466,000to,001,00am,00pm,01,012023,02,...,zoom,zoominfo,zrx,zug,zyhvz7iigr,zyuiom_ufdu,überraschung,œ100,œmost,Job Profiles
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Product Manager','Business Analyst','UX Desi..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Product Manager','Business Analyst','Marketi..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Product Manager','UX Designer','Business Ana..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Product Manager','Software Developer,'Machin..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Product Manager','Marketing Analyst','Machin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Sales Analyst','Marketing Analyst,''Business..."
368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Sales Analyst','Marketing Analyst,''Business..."
369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Sales Analyst','Marketing Analyst,''Business..."
370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['Sales Analyst','Marketing Analyst,''Business..."


In [ ]:
data_features.to_csv('/content/drive/My Drive/IR/Project/training_count_vectorizer.csv', index=False)
test_features.to_csv('/content/drive/My Drive/IR/Project/testing_count_vectorizer.csv', index=False)